In [24]:
from datasets import load_dataset
from datasets import load_from_disk
from transformers import AutoTokenizer
import torch
import os
import json
from pathlib import Path

dataset_l = load_dataset("ashaba1in/small_openwebtext")['train']
# dataset.save_to_disk('data/datasets/small_openwebtext')

In [25]:
dataset = load_from_disk('data/datasets/small_openwebtext')['train']
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-v0.1')

In [27]:
dataset_l

Dataset({
    features: ['text'],
    num_rows: 1000000
})

In [28]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1000000
})

In [18]:
def _tokenize_function(sample):
    return tokenizer(sample['text'])

In [20]:
subset = dataset.select(range(100))

subset_tok = subset.map(_tokenize_function, batched=True)

In [21]:
subset_tok['input_ids'][0][:5] # массив из токенов

[1, 4194, 28733, 581, 28733]

In [13]:
counter = 0
absolut_i = 0
seq_len = 255
for tok_text in subset_tok['input_ids']:
    i = 0
    while (start := i * seq_len) + seq_len <= len(tok_text):
        torch.save(tok_text[start: start + seq_len], f'data/datasets/extrasmall_openwebtext_tok/train/text_{absolut_i}.pt')
        i += 1
        absolut_i += 1
    
    assert len(tok_text) - start < seq_len

    torch.save(tok_text[start: ], f'data/datasets/extrasmall_openwebtext_tok/train/text_{absolut_i}.pt')
    absolut_i += 1


In [14]:
index = []
part = 'train'
data_dir = Path('data/datasets/extrasmall_openwebtext_tok')
for text_path in os.listdir(data_dir / part):
    index.append(
        {
            "text_path": str(data_dir / part / text_path)
        }
    )

index_path = Path(data_dir / 'train_index.json')

with index_path.open("w") as f:
    json.dump(index, f, indent=2)

In [48]:
index[1]

{'text_path': 'data/datasets/extrasmall_openwebtext_tok/train/text_34325.pt'}

In [3]:
tokenizer

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.1', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [12]:
tokenizer.bos_token

'<s>'

In [17]:
from torch.nn.utils.rnn import pad_sequence


a = torch.tensor([1, 2, 3, 4])
b = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8])

pad_sequence([a, b], batch_first=True).shape

torch.Size([2, 8])

In [18]:
len(tokenizer)

32000

In [20]:
torch.nn.functional.one_hot(torch.randint(1, 10, (1, 257)), num_classes=32000).shape

torch.Size([1, 257, 32000])

In [21]:
torch.load('/home/boris/Documents/ВШЭ 4 курс/nlp/hw3-llama/data/datasets/extrasmall_openwebtext_tok/train/text_0.pt', weights_only=True)

[1,
 4194,
 28733,
 581,
 28733,
 3393,
 1261,
 28725,
 382,
 1022,
 28710,
 325,
 28743,
 11348,
 28731,
 1939,
 8599,
 364,
 621,
 13980,
 28725,
 1425,
 372,
 288,
 297,
 3358,
 304,
 26517,
 2917,
 438,
 1411,
 28725,
 6758,
 13500,
 304,
 26579,
 2338,
 1753,
 477,
 264,
 1834,
 6556,
 8085,
 2125,
 1024,
 264,
 12172,
 753,
 5714,
 1918,
 25469,
 25473,
 272,
 2698,
 28725,
 3677,
 378,
 403,
 9059,
 684,
 4908,
 28723,
 13,
 13,
 1014,
 5161,
 1749,
 26656,
 11762,
 12195,
 3198,
 27092,
 308,
 3652,
 28768,
 339,
 2480,
 447,
 28708,
 390,
 272,
 865,
 6676,
 438,
 272,
 6556,
 298,
 625,
 272,
 6883,
 1059,
 272,
 2125,
 28723,
 13,
 13,
 28743,
 11348,
 12735,
 5745,
 28725,
 2818,
 356,
 17256,
 395,
 741,
 302,
 272,
 13500,
 28725,
 369,
 272,
 2969,
 16419,
 8346,
 272,
 12172,
 753,
 4205,
 330,
 313,
 304,
 10731,
 6867,
 298,
 25469,
 11931,
 28723,
 2993,
 28725,
 12172,
 753,
 11762,
 23184,
 10049,
 2404,
 930,
 420,
 1230,
 28713,
 28725,
 264,
 6676,
 693,
 403,
 

In [2]:
import torch
a = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8])

a.reshape((2, 4), )

tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])

In [7]:
from transformers import AutoModel


model_id = "dwzhu/LLaMA2-7B-PoSE-YaRN-16k"
model = AutoModel.from_pretrained(model_id)

model

config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'original_max_position_embeddings'}


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

KeyboardInterrupt: 

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

In [39]:
import torch
import torch.nn.functional as F

# Инициализируем случайные тензоры для примера
batch_size = 2
seq_len = 4
vocab_size = 10

# Логиты размером (batch_size, seq_len, vocab_size)
logits = torch.randn(batch_size, seq_len, vocab_size)

# Целевые значения (индексы) размером (batch_size, seq_len)
targets = torch.tensor([[1, 2, 0, 3], [4, 5, 6, 0]])

# Маска размером (batch_size, seq_len), где 1 указывает на значимые позиции
mask = torch.tensor([[1, 1, 0, 0], [1, 1, 1, 0]], dtype=torch.float32)

# Переформатируем логиты для CrossEntropyLoss
# Из (batch_size, seq_len, vocab_size) в ((batch_size * seq_len), vocab_size)
logits = logits.view(-1, vocab_size)

# Переформатируем целевые значения из (batch_size, seq_len) в ((batch_size * seq_len),)
targets = targets.view(-1)

# Вычисляем CrossEntropyLoss для всех позиций
loss = F.cross_entropy(logits, targets)

In [40]:
logits.dtype

torch.float32

In [42]:
tokenizer.decode([1, 2, 3, 4])

'<s></s>\x00\x01'